### 데이터 받아오기

In [1]:
import json
import numpy as np
import random
import nltk
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

In [2]:
with open("data/reviews_Books_5.json","r") as data_file:
    raw_data=data_file.readlines()

In [3]:
json_list = []
for i in range(len(raw_data)):
    try:
        json_data = json.loads(raw_data[i])
        json_list.append(json_data)
    except:
        print(raw_data[i],i)

{"reviewerID": "AAXUNK0W2DZG5", "asin": "0060520841", "reviewerName": "Amazon Customer \"leneker\"", "helpful": [5, 10], "reviewText": "1996 Bernard Goldberg wrote an editorial for the Wall Street Journal that said there was an obvious bias on the part of network new shows  for the liberal point of view.  he then illustrated this with an example that he dissected in-depth.  The reaction to this observation was the ruination of her career, and the beginning of his status as a pariah among most newsmen.  This book is used to add more ammo to the controversy.That the journalists who so eagerly pry into other peoples lives and business should be reluctant to be examined is hardly surprising.  Almost no one really wants to have his life or business dissected by Mike Wallace not even Dan Rather.  Some facts in this book are really potent such as the survey results which show how the average journalist and the average American are often vastly at odds.  Other chapters highlight different stor

### 데이터 분리하기

In [4]:
review_data = []
for i in range(len(json_list)):
    if (len(json_list[i]["reviewText"])!=0):
        review_dict={}
        review_dict["reviewText"] = json_list[i]["reviewText"]
        review_dict["overall"] = json_list[i]["overall"]
        review_data.append(review_dict)

In [5]:
import random
random_data = random.sample(review_data,len(review_data))


In [6]:
reviewText = []
for i in range(len(random_data)):
    reviewText.append(random_data[i]["reviewText"])

In [7]:
y_data = []
for i in range(len(random_data)):
    y_data.append(random_data[i]["overall"])

In [8]:
from collections import Counter
Counter(y_data)

Counter({1.0: 4239, 2.0: 5102, 3.0: 10176, 4.0: 21555, 5.0: 52105})

<h3>데이터 전처리</h3>

In [9]:
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re

In [10]:
def review_to_words(raw_review):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    
   
    cleaned_text_list=[]
    for i in range(len(raw_review)):
        
        review_text = BeautifulSoup(raw_review[i],"html.parser").get_text()
        letters_only = re.sub("[^a-zA-Z]", " ", review_text)
        words = letters_only.lower().split()
        cleaned_text = " ".join(words)
        cleaned_text_list.append(cleaned_text)

    return cleaned_text_list

       

In [12]:
pre_data=review_to_words(reviewText)

### 가장 작은 평점으로 갯수 맞추기

In [13]:
from collections import Counter
length=sorted(Counter(y_data).most_common())[0][1]

In [14]:
search=[1.0,2.0,3.0,4.0,5.0]
count=0
text=[]
y=[]

for i in search:
    count=0
    for index,value in enumerate(y_data):
        if (count==length): break
        elif (i==value):
            text.append(pre_data[index])
            y.append(value)
            count=count+1

In [15]:
Counter(y)

Counter({1.0: 4239, 2.0: 4239, 3.0: 4239, 4.0: 4239, 5.0: 4239})

<h3>데이터 셔플 다시</h3>

In [16]:
c = list(zip(text, y))
random.shuffle(c)

In [17]:
a, b = zip(*c)

In [18]:
reviewText=list(a)
overall=np.array(list(b))

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True,ngram_range=(1,2),lowercase=True,min_df=2,max_df=0.2,stop_words='english')
tfidf = vectorizer.fit_transform(reviewText)

In [36]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tfidf , overall, test_size=0.33, random_state=42)

In [37]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((14200, 180833), (14200,), (6995, 180833), (6995,))

### 예측

### LogisticRegression

In [52]:
from sklearn import linear_model
#logreg = linear_model.LogisticRegression()
logreg = linear_model.LogisticRegression(random_state=42,solver='sag',multi_class='multinomial',warm_start=True)
logreg.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=42, solver='sag',
          tol=0.0001, verbose=0, warm_start=True)

In [53]:
from sklearn.metrics import accuracy_score
accuracy_score(logreg.predict(x_test), y_test)

0.47891350964974982

In [54]:
from sklearn.metrics import precision_score
precision_score(logreg.predict(x_test), y_test,average=None) 
#e scores for each class are returned. Otherwise, this determines the type of averaging performed on the data:

array([ 0.58134786,  0.3995649 ,  0.35785714,  0.41891892,  0.63758389])

In [55]:
from sklearn.metrics import classification_report
print(classification_report(y_test, logreg.predict(x_test)))

             precision    recall  f1-score   support

        1.0       0.58      0.58      0.58      1469
        2.0       0.42      0.40      0.41      1379
        3.0       0.42      0.36      0.39      1400
        4.0       0.43      0.42      0.42      1406
        5.0       0.51      0.64      0.57      1341

avg / total       0.47      0.48      0.47      6995



In [56]:
from sklearn import metrics
for i in range(1,6):
    fpr, tpr, thresholds = metrics.roc_curve(y_test, logreg.predict(x_test), pos_label=i)
#pos_label Label considered as positive and others are considered negative.
    print(i,"점 = ",metrics.auc(fpr, tpr))

1 점 =  0.190553068396
2 점 =  0.32541386983
3 점 =  0.495523043534
4 점 =  0.68955384319
5 점 =  0.81780474159


### INPUT 

In [43]:
texts = ["""I enjoyed this book, as long as I kept in mind that the book was written for young adults.  
         The characters were believable and it was easy to identify with their pain.""",
        "I think the author gets lost in his own thoughts and trys to make storys of something that's not there.",
        """I was very excited to start reading this book and although I did like it, 
        I thought that a lot of the words to describe body parts was kind of high-schoolish.  
        Didn't care for the cliff hanging ending.""",
        """I don't know if you are familiar with FANFIC on the internet but that what this book reminded me of.  
        I love the series and thought I would like the books...not so much.""",
        """As I love western era romances, this one was wonderful.  An easy read and ended the way we always hope....happily!"""]
overall=[4,1,3,2,5]
vecs = vectorizer.transform(texts) 

In [44]:
print(overall,logreg.predict(vecs))

[4, 1, 3, 2, 5] [ 4.  1.  2.  5.  5.]


In [28]:
print(y_test[:15],logreg.predict(x_test)[:15])

[ 5.  1.  4.  4.  5.  4.  3.  1.  1.  1.  1.  2.  5.  4.  4.] [ 5.  1.  4.  3.  5.  4.  3.  2.  2.  2.  2.  3.  4.  3.  5.]
